In [2]:
# Instalación de paquetes necesarios
!pip install scikit-learn torch torchvision pycocotools pillow

In [21]:
# Librerías
import os
import numpy as np
from PIL import Image
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.utils import draw_segmentation_masks
import matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import os
from PIL import Image
import numpy as np
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torch.utils.data import Dataset, DataLoader, Subset
from pycocotools.coco import COCO
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Configuración de dispositivo (GPU si está disponible, sino CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


In [5]:
# Definición de rutas
coco_file_path = r'C:\Users\HENRY\Documents\Seed_Project\Project_germination\result.json'
image_dir = r'C:\Users\HENRY\Documents\Seed_Project\Project_germination\images'

In [6]:
# Cargar las anotaciones COCO
coco = COCO(coco_file_path)

loading annotations into memory...
Done (t=1.05s)
creating index...
index created!


In [7]:
# Obtener todos los IDs de las imágenes
img_ids = list(coco.imgs.keys())

In [8]:
# Dividir los datos: 70% entrenamiento, 20% validación, 10% prueba
train_ids, temp_ids = train_test_split(img_ids, test_size=0.3, random_state=42)
val_ids, test_ids = train_test_split(temp_ids, test_size=0.33, random_state=42)

print(f"Total imágenes: {len(img_ids)}")
print(f"Imágenes de entrenamiento: {len(train_ids)} ({len(train_ids)/len(img_ids)*100:.2f}%)")
print(f"Imágenes de validación: {len(val_ids)} ({len(val_ids)/len(img_ids)*100:.2f}%)")
print(f"Imágenes de prueba: {len(test_ids)} ({len(test_ids)/len(img_ids)*100:.2f}%)")

Total imágenes: 159
Imágenes de entrenamiento: 111 (69.81%)
Imágenes de validación: 32 (20.13%)
Imágenes de prueba: 16 (10.06%)


In [15]:
# Verificación de las rutas
if not os.path.exists(coco_file_path):
    print(f"Error: No se encuentra el archivo JSON en: {coco_file_path}")
if not os.path.exists(image_dir):
    print(f"Error: No se encuentra el directorio de imágenes en: {image_dir}")

In [16]:
# Verificación explícita de las rutas
import os

def verificar_ruta(ruta, tipo):
    if os.path.exists(ruta):
        print(f"✅ El {tipo} existe en: {ruta}")
        return True
    else:
        print(f"❌ Error: No se encuentra el {tipo} en: {ruta}")
        return False

# Verificar el archivo JSON
json_existe = verificar_ruta(coco_file_path, "archivo JSON")

# Verificar el directorio de imágenes
imagenes_existen = verificar_ruta(image_dir, "directorio de imágenes")

if json_existe and imagenes_existen:
    print("🎉 Todas las rutas son correctas. Puedes proceder con tu modelo.")
else:
    print("⚠️ Hay problemas con las rutas. Por favor, verifica y corrige antes de continuar.")

✅ El archivo JSON existe en: C:\Users\HENRY\Documents\Seed_Project\Project_germination\result.json
✅ El directorio de imágenes existe en: C:\Users\HENRY\Documents\Seed_Project\Project_germination\images
🎉 Todas las rutas son correctas. Puedes proceder con tu modelo.


In [17]:
# Definición del conjunto de datos personalizado
class SeedDataset(Dataset):
    def __init__(self, coco, image_dir, transforms=None):
        self.coco = coco
        self.image_dir = image_dir
        self.transforms = transforms
        self.ids = list(self.coco.imgs.keys())

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)
        
        img_info = self.coco.loadImgs(img_id)[0]
        img_path = os.path.join(self.image_dir, img_info['file_name'])
        img = Image.open(img_path).convert("RGB")
        
        num_objs = len(anns)
        boxes = []
        masks = []
        for i in range(num_objs):
            xmin = anns[i]['bbox'][0]
            ymin = anns[i]['bbox'][1]
            xmax = xmin + anns[i]['bbox'][2]
            ymax = ymin + anns[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
            mask = self.coco.annToMask(anns[i])
            masks.append(mask)
        
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        masks = torch.as_tensor(masks, dtype=torch.uint8)
        
        image_id = torch.tensor([img_id])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)
        
        target = {}
        target["boxes"] = boxes
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        
        return img, target

    def __len__(self):
        return len(self.ids)

In [19]:
# Función para obtener el modelo
def get_model(num_classes):
    model = maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, num_classes)
    return model

In [20]:
# Preparación de los datos
coco = COCO(coco_file_path)
dataset = SeedDataset(coco, image_dir)
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.39s)
creating index...
index created!


In [22]:
# Función para seleccionar un subconjunto aleatorio de imágenes
def get_subset_indices(dataset, num_images):
    return torch.randperm(len(dataset))[:num_images].tolist()

In [23]:
# Función para visualizar y exportar máscaras
def visualize_and_export_masks(model, dataset, indices, output_dir):
    model.eval()
    device = next(model.parameters()).device
    
    os.makedirs(output_dir, exist_ok=True)
    
    for idx in indices:
        image, target = dataset[idx]
        image_tensor = torchvision.transforms.functional.to_tensor(image).unsqueeze(0).to(device)
        
        with torch.no_grad():
            prediction = model(image_tensor)[0]
        
        image_np = image.numpy().transpose((1, 2, 0))
        masks = prediction['masks'].cpu().numpy()
        
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1)
        plt.imshow(image_np)
        plt.title('Original Image')
        
        plt.subplot(1, 3, 2)
        plt.imshow(image_np)
        for mask in masks:
            plt.imshow(mask[0], alpha=0.5, cmap='jet')
        plt.title('Predicted Masks')
        
        plt.subplot(1, 3, 3)
        plt.imshow(image_np)
        for mask in target['masks']:
            plt.imshow(mask.numpy(), alpha=0.5, cmap='jet')
        plt.title('Ground Truth Masks')
        
        plt.savefig(os.path.join(output_dir, f'mask_comparison_{idx}.png'))
        plt.close()

In [25]:
# Dividir los datos en conjuntos de entrenamiento, validación y prueba
train_indices, temp_indices = train_test_split(range(len(dataset)), test_size=0.3, random_state=42)
val_indices, test_indices = train_test_split(temp_indices, test_size=0.33, random_state=42)

train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, collate_fn=lambda x: tuple(zip(*x)))

print(f"Tamaño del conjunto de entrenamiento: {len(train_dataset)}")
print(f"Tamaño del conjunto de validación: {len(val_dataset)}")
print(f"Tamaño del conjunto de prueba: {len(test_dataset)}")

NameError: name 'batch_size' is not defined